In [2]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 98.2 MB/s eta 0:0

In [3]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

import os
import pandas as pd
import pathlib
import chromadb
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

os.environ["OPENAI_API_KEY"] = ""

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [5]:
import glob

In [7]:
instruction = "Represent the podcast transcript document for retrieval: "

# read transcript data
path = 'transcripts-0[0-9].csv'
transcripts_df = pd.read_csv(path, index_col=0)



# Create list of instruction - transcript pairs (100 first episodes)
transcripts_df  = transcripts_df.head(100)
texts_with_instructions = []
for index, row in transcripts_df.iterrows():
    texts_with_instructions.append([instruction, row["transcript"]])

# calculate embeddings (100 first episodes took about 6 min)
model = INSTRUCTOR('hkunlp/instructor-large')
customized_embeddings = model.encode(texts_with_instructions)

load INSTRUCTOR_Transformer
max_seq_length  512


In [8]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'test-db'

# format data that if can be put to the chroma collection
embeddings = customized_embeddings.tolist()
documents = transcripts_df["transcript"].tolist()
ids = transcripts_df.index.astype(str).to_list()
metadata = transcripts_df[['episode_description']].to_dict(orient='records')

#loading into chroma
client = chromadb.PersistentClient(path=persist_directory)
# create the collection and add documents
collection = client.create_collection("transcripts", embedding_function=model)
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadata,
    ids=ids,
)

In [21]:
# Try querying the database
query_texts=[['Represent the statement for retrieving podcast documents: ',"I'm looking for podcasts about sports and especially basketball"]]
query_embedding = model.encode(query_texts).tolist()
results = collection.query(
    query_embeddings=query_embedding,
    n_results=1 #only using the first result for now
)

In [37]:
print(results)

{'ids': [['43']], 'distances': [[0.3402264714241028]], 'metadatas': [[{'episode_description': "Today we have a Special announcements We do our Deep Dive on the last Character Octane and So many people are claiming that Apex is dead or Dying. Let's cover all sides of the argument and give our opinions. Facebook & YouTube - Threat X3 Productions Twitter - @TX3Productions threatx3productions@gmail.com Twitter- Apex Legends HUBcast @ApexLegendscast Discord- Threat X3 Productions https://discord.gg/nsjnv4d  ---   This episode is sponsored by  · Anchor: The easiest way to make a podcast.  https://anchor.fm/app "}]], 'embeddings': None, 'documents': [["Hey guys, this is Nick before we got started with the show proper. We wanted to share something with you that we've been working on enjoy. Hello, I'm Nick and I'm Nate. We are threatened X3 Productions. Tell me if this sounds familiar. I used to listen to radio all the time either driving or on in the background at home. But after a while, I st

In [23]:
if results:
    # Extract and print the output
    output = results['documents'] 
    print("Query Result:")
    print(output)
else:
    print("No results found.")

Query Result:
[["Hey guys, this is Nick before we got started with the show proper. We wanted to share something with you that we've been working on enjoy. Hello, I'm Nick and I'm Nate. We are threatened X3 Productions. Tell me if this sounds familiar. I used to listen to radio all the time either driving or on in the background at home. But after a while, I started listening less and less the DJ's just didn't talk about things that I was interested in mainly gossip like who's getting married who cheated on.  Who and oh, is that a baby bump but as a super nerve there was just no topics. I liked hearing about movies were mainly good or bad. No in-depth talks and video games were the worst if they were brought up at all. It was almost guaranteed to be negative. I didn't really mind the music brakes but the ad seemed to go on forever. So I switched to podcasts. I got the topics that I cared about and adds didn't overstay their welcome. So what if we could have the best of both worlds, tha

In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [32]:
import re
import string
import nltk
from nltk.corpus import stopwords

# Function to clean and preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    stop_words = set(stopwords.words('english'))
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_words)


    return text


# Preprocess the text
text = output[0][0]
cleaned_text = preprocess_text(text)

# Print the cleaned text
print(cleaned_text)

hey guys this is nick before we got started with the show proper we wanted to share something with you that weve been working on enjoy hello im nick and im nate we are threatened x productions tell me if this sounds familiar i used to listen to radio all the time either driving or on in the background at home but after a while i started listening less and less the djs just didnt talk about things that i was interested in mainly gossip like whos getting married who cheated on who and oh is that a baby bump but as a super nerve there was just no topics i liked hearing about movies were mainly good or bad no indepth talks and video games were the worst if they were brought up at all it was almost guaranteed to be negative i didnt really mind the music brakes but the ad seemed to go on forever so i switched to podcasts i got the topics that i cared about and adds didnt overstay their welcome so what if we could have the best of both worlds thats where we come in introducing the national ne

In [29]:
!pip install transformers


In [30]:
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
# Load the pre-trained BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [40]:
# summarization function
def generate_summary(text, max_length=500, min_length=100):
    input_text = text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=1024)

    # Generate the summary
    summary_ids = model.generate(inputs["input_ids"], max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Generating the summary
generated_summary = generate_summary(cleaned_text)
print("Generated Summary:", generated_summary)

Generated Summary: Nerdy net radio direct or nerd radio for short topics you love from podcast but music curated by the community and best of all no ads only short sponsor reads. If you like what you hear and you would like to support us you can help us make this dream a reality nerd radio youre stopped for music video games movies and so much more alright guys thats our big announcement hope you are excited as excited as we are to bring this to you now lets get started with the show welcome legends to the hub cast your central hub for everything apex.


In [41]:
#download as text file
with open("summary1.txt", "w") as f:
    f.write("".join(generated_summary))